In [10]:
import json 
import numpy as np 
import tensorflow as tf
import keras
from keras import layers, models
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalAveragePooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [11]:
with open('intents.json') as file:
    data = json.load(file)

In [12]:
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [13]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)


In [14]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) # adding out of vocabulary token
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [15]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d_1  (None, 16)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_3 (Dense)             (None, 16)                272       
                                                                 
 dense_4 (Dense)             (None, 16)                272       
                                                                 
 dense_5 (Dense)             (None, 79)                1343      
                                                                 
Total params: 17887 (69.87 KB)
Trainable params: 17887 (69.87 KB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [17]:
epochs = 550
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/550
9/9 [==============================] - 2s 3ms/step - loss: 4.3696 - accuracy: 0.0106 
Epoch 2/550
9/9 [==============================] - 0s 3ms/step - loss: 4.3666 - accuracy: 0.0318
Epoch 3/550
9/9 [==============================] - 0s 3ms/step - loss: 4.3642 - accuracy: 0.0353
Epoch 4/550
9/9 [==============================] - 0s 3ms/step - loss: 4.3620 - accuracy: 0.0459
Epoch 5/550
9/9 [==============================] - 0s 3ms/step - loss: 4.3590 - accuracy: 0.0495
Epoch 6/550
9/9 [==============================] - 0s 4ms/step - loss: 4.3556 - accuracy: 0.0565
Epoch 7/550
9/9 [==============================] - 0s 5ms/step - loss: 4.3522 - accuracy: 0.0495
Epoch 8/550
9/9 [==============================] - 0s 4ms/step - loss: 4.3469 - accuracy: 0.0495
Epoch 9/550
9/9 [==============================] - 0s 4ms/step - loss: 4.3418 - accuracy: 0.0495
Epoch 10/550
9/9 [==============================] - 0s 4ms/step - loss: 4.3356 - accuracy: 0.0495
Epoch 11/550
9/9 [==========

In [18]:
# saving model
model.save("chat_model")

import pickle

# saving tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

# saving label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model\assets


INFO:tensorflow:Assets written to: chat_model\assets
